In [1]:
# Huggingface: Qwen/Qwen2-7B-Instruct-GPTQ-Int8
model_qwen2 = r"/mnt/workspace/model/Qwen2-7B-Instruct-GPTQ-Int8"
!ls /mnt/workspace/model/Qwen2-7B-Instruct-GPTQ-Int8

config.json		model-00001-of-00003.safetensors  README.md
generation_config.json	model-00002-of-00003.safetensors  tokenizer_config.json
LICENSE			model-00003-of-00003.safetensors  tokenizer.json
merges.txt		model.safetensors.index.json	  vocab.json


In [2]:
from modelscope import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    model_qwen2,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_qwen2)

2024-07-24 14:48:29,769 - modelscope - INFO - PyTorch version 2.3.0+cu121 Found.
2024-07-24 14:48:29,774 - modelscope - INFO - TensorFlow version 2.16.1 Found.
2024-07-24 14:48:29,774 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2024-07-24 14:48:30,384 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 03830e53bae2c0a8f8ac72c969ac6ac1 and a total number of 980 components indexed
/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-24 14:48:33.919990: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
20

In [3]:
# 推理示例1：一步步进行
prompt = "香蕉是什么颜色的？"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


香蕉通常是黄色的，但未成熟的香蕉可能是绿色的。有些品种的香蕉可能在成熟过程中会呈现出不同的颜色变化，例如红色或黑色。此外，香蕉皮的颜色也可能因为保存条件、品种和成熟度等因素而有所不同。


In [6]:
# 推理示例2：简洁推理
from transformers import pipeline
from modelscope.utils.constant import Tasks
generator = pipeline(Tasks.text_generation, model=model, tokenizer=tokenizer)
result = generator(messages, max_length=5000)[0]['generated_text']
print(result)

[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': '香蕉是什么颜色的？'}, {'role': 'assistant', 'content': '香蕉通常是黄色的，但也可能是绿色或红色的，这取决于它们的成熟度。未成熟的香蕉是绿色的，而部分成熟的香蕉则是绿色带黄色。完全成熟的香蕉则呈现出鲜亮的黄色。有些品种的香蕉在成熟时会变成红色。'}]
